<a href="https://colab.research.google.com/github/NewTAs/GoogLeNet/blob/main/GoogLeNet_Step5_DriveVer_Epoch100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step1 Code
import os
import re
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

from google.colab import drive
drive.mount('googlenet')

# New Cord
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

#np.random.seed(4) # 랜덤시드 고정시키기

Mounted at googlenet


In [2]:
pictureWeightHeightSize = 227

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TrainData_Step5/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),
        batch_size=10,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TestData_Step5/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),    
        batch_size=10,
        class_mode='categorical')

Found 300 images belonging to 10 classes.
Found 20 images belonging to 10 classes.


In [3]:
input_layer = tf.keras.layers.Input(shape=(pictureWeightHeightSize, pictureWeightHeightSize, 3))

In [4]:
stem_layer = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(input_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)
stem_layer = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(stem_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)

In [5]:
inception_block3a_1x1 = tf.keras.layers.Conv2D(32, (1, 1), strides=(1, 1), padding='SAME', activation='relu')(stem_layer)

inception_block3a_3x3 = tf.keras.layers.Conv2D(48, (1, 1), padding='SAME', activation='relu')(stem_layer)
inception_block3a_3x3 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME', activation='relu')(inception_block3a_3x3)

inception_block3a_5x5 = tf.keras.layers.Conv2D(8, (1, 1), padding='SAME', activation='relu')(stem_layer)
inception_block3a_5x5 = tf.keras.layers.Conv2D(16, (5, 5), padding='SAME', activation='relu')(inception_block3a_5x5)

inception_block3a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(stem_layer)
inception_block3a_pool = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(inception_block3a_pool)

inception_block3a = tf.keras.layers.Concatenate()([inception_block3a_1x1, inception_block3a_3x3,
                                                   inception_block3a_5x5, inception_block3a_pool]) # 32x32x128

In [6]:
# inception_block3b Here on Original Inception
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block3a) # 16x16x128

In [7]:
inception_block4a_1x1 = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block4a_3x3 = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_3x3 = tf.keras.layers.Conv2D(96, (3, 3), padding='SAME', activation='relu')(inception_block4a_3x3)

inception_block4a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_5x5 = tf.keras.layers.Conv2D(48, (5, 5), padding='SAME', activation='relu')(inception_block4a_5x5)

inception_block4a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block4a_pool = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block4a_pool)

inception_block4a = tf.keras.layers.Concatenate()([inception_block4a_1x1, inception_block4a_3x3,
                                                   inception_block4a_5x5, inception_block4a_pool]) # 16x16x240

# inception_block4b Here on Original Inception
# inception_block4c Here on Original Inception
# inception_block4d Here on Original Inception
# inception_block4e Here on Original Inception

In [8]:
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block4a) # 8x8x240

In [9]:
inception_block5a_1x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block5a_3x3 = tf.keras.layers.Conv2D(80, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_3x3 = tf.keras.layers.Conv2D(120, (3, 3), padding='SAME', activation='relu')(inception_block5a_3x3)

inception_block5a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block5a_5x5)

inception_block5a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block5a_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block5a_pool)

inception_block5a = tf.keras.layers.Concatenate()([inception_block5a_1x1, inception_block5a_3x3,
                                                   inception_block5a_5x5, inception_block5a_pool]) # 8x8x416
# inception_block5b Here on Original Inception

In [10]:
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inception_block5a) # 416
dropout = tf.keras.layers.Dropout(0.4)(avg_pool)
out_layer = tf.keras.layers.Dense(10, activation='softmax')(dropout)

In [11]:
model = tf.keras.models.Model(input_layer, out_layer)

#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 227, 227, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 227, 227, 64) 18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 227, 227, 48) 3120        conv2d_1[0][0]                   
______________________________________________________________________________________________

In [12]:
model.fit(train_generator, epochs=100, validation_data=test_generator)

Epoch 1/100
30/30 [==============================] - 348s 11s/step - loss: 2.2627 - accuracy: 0.1333 - val_loss: 2.0474 - val_accuracy: 0.2000
Epoch 2/100
30/30 [==============================] - 333s 11s/step - loss: 1.7053 - accuracy: 0.3233 - val_loss: 1.3699 - val_accuracy: 0.6000
Epoch 3/100
30/30 [==============================] - 331s 11s/step - loss: 0.9391 - accuracy: 0.6400 - val_loss: 0.3437 - val_accuracy: 1.0000
Epoch 4/100
30/30 [==============================] - 329s 11s/step - loss: 0.6670 - accuracy: 0.7600 - val_loss: 0.3239 - val_accuracy: 0.9000
Epoch 5/100
30/30 [==============================] - 333s 11s/step - loss: 0.4012 - accuracy: 0.8533 - val_loss: 0.2978 - val_accuracy: 0.9000
Epoch 6/100
30/30 [==============================] - 333s 11s/step - loss: 0.1324 - accuracy: 0.9700 - val_loss: 0.0300 - val_accuracy: 1.0000
Epoch 7/100
30/30 [==============================] - 333s 11s/step - loss: 0.1657 - accuracy: 0.9533 - val_loss: 0.2734 - val_accuracy: 0.9000

In [13]:
print("-- Evaluate --")
scores = model.evaluate(test_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
2/2 [==============================] - 5s 2s/step - loss: 3.2516e-04 - accuracy: 1.0000
accuracy: 100.00%


In [14]:
print("-- Predict --")
output = model.predict(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'1': 0, '10': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
[[0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.002 0.997 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000]
 [0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.002 0.997 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000]
 [0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
 [1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.